In [20]:
import numpy as np 
import math
import pandas as pd # 
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from components.data_pipeline import dataPipeline

import warnings
warnings.filterwarnings('ignore')

In [2]:
#TESTES
df_test = pd.read_csv('input/teste1.csv')
df_test, _ = dataPipeline(df_test)
df_test = df_test.sort_values(by='data_semana_comercial')

df_test2 = pd.read_csv('input/teste2.csv')
df_test2, _ = dataPipeline(df_test2)
df_test2 = df_test2.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado
Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


In [3]:
#TREINO
df = pd.read_csv('input/treino.csv')
df, _ = dataPipeline(df)
df = df.sort_values(by='data_semana_comercial')

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


# Prophet

In [4]:
df_prophet = df.copy()
df_prophet.data_semana_comercial = pd.to_datetime(df_prophet.data_semana_comercial)
df_prophet_406 = df_prophet[df_prophet.cod_loja == 406]
df_prophet_407 = df_prophet[df_prophet.cod_loja == 407]
df_prophet_406.drop('cod_loja', axis=1, inplace=True)
df_prophet_407.drop('cod_loja', axis=1, inplace=True)
df_prophet_406.columns = ['ds', 'y']
df_prophet_407.columns = ['ds', 'y']

In [5]:
model_406 = Prophet()
model_406.fit(df_prophet_406)

model_407 = Prophet()
model_407.fit(df_prophet_407)

12:51:20 - cmdstanpy - INFO - Chain [1] start processing
12:51:20 - cmdstanpy - INFO - Chain [1] done processing
12:51:20 - cmdstanpy - INFO - Chain [1] start processing
12:51:20 - cmdstanpy - INFO - Chain [1] done processing


In [6]:
df_prophet_test_406 = df_test.copy()
df_prophet_test_406 = df_prophet_test_406[df_prophet_test_406.cod_loja == 406]
df_prophet_test_406.drop('cod_loja', axis=1, inplace=True)
df_prophet_test_406.data_semana_comercial = pd.to_datetime(df_prophet_test_406.data_semana_comercial)
df_prophet_test_406.columns = ['ds', 'venda']
df_prophet_test_406_pred = df_prophet_test_406.drop('venda', axis=1).copy()

df_prophet_test_407 = df_test.copy()
df_prophet_test_407 = df_prophet_test_407[df_prophet_test_407.cod_loja == 407]
df_prophet_test_407.drop('cod_loja', axis=1, inplace=True)
df_prophet_test_407.data_semana_comercial = pd.to_datetime(df_prophet_test_407.data_semana_comercial)
df_prophet_test_407.columns = ['ds', 'venda']
df_prophet_test_407_pred = df_prophet_test_407.drop('venda', axis=1).copy()

df_prophet_test2_406 = df_test2.copy()
df_prophet_test2_406 = df_prophet_test2_406[df_prophet_test2_406.cod_loja == 407]
df_prophet_test2_406.drop('cod_loja', axis=1, inplace=True)
df_prophet_test2_406.data_semana_comercial = pd.to_datetime(df_prophet_test2_406.data_semana_comercial)
df_prophet_test2_406.columns = ['ds', 'venda']
df_prophet_test2_406_pred2 = df_prophet_test2_406.drop('venda', axis=1).copy()

df_prophet_test2_407 = df_test2.copy()
df_prophet_test2_407 = df_prophet_test2_407[df_prophet_test2_407.cod_loja == 407]
df_prophet_test2_407.drop('cod_loja', axis=1, inplace=True)
df_prophet_test2_407.data_semana_comercial = pd.to_datetime(df_prophet_test2_407.data_semana_comercial)
df_prophet_test2_407.columns = ['ds', 'venda']
df_prophet_test2_407_pred2 = df_prophet_test2_407.drop('venda', axis=1).copy()

In [7]:
forecast1_406 = model_406.predict(df_prophet_test_406_pred)
forecast2_406 = model_406.predict(df_prophet_test2_406_pred2)

forecast1_407 = model_407.predict(df_prophet_test_406_pred)
forecast2_407 = model_407.predict(df_prophet_test2_407_pred2)

In [8]:
forecast2_406.loc[forecast2_406.yhat < 0, 'yhat'] = 0
forecast2_407.loc[forecast2_407.yhat < 0, 'yhat'] = 0

In [9]:
forecast1_406 = forecast1_406[['ds', 'yhat']]
forecast2_406 = forecast2_406[['ds', 'yhat']]
forecast1_407 = forecast1_407[['ds', 'yhat']]
forecast2_407 = forecast2_407[['ds', 'yhat']]

In [10]:
forecast1_406['cod_loja'] = 406
forecast2_406['cod_loja'] = 406
forecast1_407['cod_loja'] = 407
forecast2_407['cod_loja'] = 407

In [11]:
forecast1_406

,ds,yhat,cod_loja
0,2022-05-01,364.871880,406
1,2022-05-08,557.713960,406
2,2022-05-15,848.799900,406
3,2022-05-22,1070.398502,406
4,2022-05-29,1088.237261,406
5,2022-06-05,937.359420,406
6,2022-06-12,791.239217,406
7,2022-06-19,789.025004,406


In [12]:
df_merged_406 = df_test.merge(forecast1_406, left_on=['data_semana_comercial', 'cod_loja'], right_on=['ds', 'cod_loja'])
df_merged_406.drop('ds', axis=1, inplace=True)
df_merged2_406 = df_test2.merge(forecast2_406, left_on=['data_semana_comercial', 'cod_loja'], right_on=['ds', 'cod_loja'])
df_merged2_406.drop('ds', axis=1, inplace=True)

df_merged_407 = df_test.merge(forecast1_407, left_on=['data_semana_comercial', 'cod_loja'], right_on=['ds', 'cod_loja'])
df_merged_407.drop('ds', axis=1, inplace=True)
df_merged2_407 = df_test2.merge(forecast2_407, left_on=['data_semana_comercial', 'cod_loja'], right_on=['ds', 'cod_loja'])
df_merged2_407.drop('ds', axis=1, inplace=True)


In [21]:
# Concatenando os DataFrames verticalmente
df_concatenated = pd.concat([df_merged_406, df_merged2_406, df_merged_407, df_merged2_407])

# Reinicializando os índices do novo DataFrame
df_concatenated = df_concatenated.reset_index(drop=True)
df_concatenated['yhat'] = df_concatenated['yhat'].apply(lambda x: math.floor(x))

In [22]:
df_concatenated['APE'] = abs((df_concatenated['venda'] - df_concatenated['yhat']) / df_concatenated['venda']) * 100

In [28]:
df_concatenated['APE'] = df_concatenated['APE'].round(2)

In [30]:
df_concatenated.to_csv('results/web/web_forecasting_prophet.csv', index=False, sep=';')